In [1]:
# Import libary and sklearn
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import export_text

In [2]:
path = '/Users/josephzhuo/anaconda_projects/eg-online-sales-analysis'

df_details = pd.read_csv(path + '/Details.csv')
df_orders = pd.read_csv(path + '/Orders.csv')

In [3]:
# Display descriptive statistics
print("\nDescriptive Statistics:")
print(df_details.describe())


Descriptive Statistics:
            Amount      Profit     Quantity
count  1500.000000  1500.00000  1500.000000
mean    291.847333    24.64200     3.743333
std     461.924620   168.55881     2.184942
min       4.000000 -1981.00000     1.000000
25%      47.750000   -12.00000     2.000000
50%     122.000000     8.00000     3.000000
75%     326.250000    38.00000     5.000000
max    5729.000000  1864.00000    14.000000


In [4]:
# Display descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df_orders.describe())


Descriptive Statistics:
       Order ID  Order Date CustomerName        State    City
count       500         500          500          500     500
unique      500         307          336           19      25
top     B-25799  24-11-2018       Shreya  Maharashtra  Indore
freq          1           7            6           94      71


In [5]:
# 1) Convert categorical columns to numerical (Label)
cat_map  = {'Clothing': 0, 'Electronics': 1, 'Furniture': 2}
sub_map  = {'Accessories': 0, 'Bookcases': 1, 'Chairs': 2, 'Electronic Games': 3, 'Furnishings': 4,
            'Hankerchief': 5, 'Kurti': 6, 'Leggings': 7, 'Phones': 8, 'Printers': 9,
            'Saree': 10, 'Shirt': 11, 'Skirt': 12, 'Stole': 13, 'T-shirt': 14, 'Tables': 15, 'Trousers': 16}
pay_map  = {'COD': 0, 'Credit Card': 1, 'Debit Card': 2, 'EMI': 3, 'UPI': 4}

df_enc = df_details.copy()
df_enc['Category']      = df_enc['Category'].map(cat_map)
df_enc['Sub-Category']  = df_enc['Sub-Category'].map(sub_map)
df_enc['PaymentMode']   = df_enc['PaymentMode'].map(pay_map)

In [6]:
X = df_enc[['Quantity','Category','Sub-Category','PaymentMode']].astype(float)
y = df_enc['Profit'].astype(float)

In [7]:
# Training and tst split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# DecisionTreeRegressor
param_grid = {'max_depth': [3,5,7,10], 'min_samples_split': [2,5,10], 'min_samples_leaf': [1,2,4]}
grid = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train_scaled, y_train)
best_model = grid.best_estimator_

# Export Tree Values for PBI
feature_names = ['Quantity','Category','Sub-Category','PaymentMode']  
rules = export_text(best_model, feature_names=feature_names)
print(rules)

# Print Scaler
print("MEAN:", scaler.mean_) 
print("SCALE:", scaler.scale_) 


|--- Sub-Category <= -1.42
|   |--- Quantity <= 3.06
|   |   |--- Quantity <= 0.34
|   |   |   |--- value: [58.53]
|   |   |--- Quantity >  0.34
|   |   |   |--- value: [180.50]
|   |--- Quantity >  3.06
|   |   |--- value: [-1307.00]
|--- Sub-Category >  -1.42
|   |--- Quantity <= 3.97
|   |   |--- Sub-Category <= 1.72
|   |   |   |--- value: [14.08]
|   |   |--- Sub-Category >  1.72
|   |   |   |--- value: [96.28]
|   |--- Quantity >  3.97
|   |   |--- PaymentMode <= -0.62
|   |   |   |--- value: [936.50]
|   |   |--- PaymentMode >  -0.62
|   |   |   |--- value: [151.50]

MEAN: [3.76       0.53333333 7.83583333 1.50833333]
SCALE: [2.20243804 0.75755455 4.46081636 1.62529912]



# Power BI In‑Notebook Notes — ML Predicted Profit

---

## 1) What this does
We **reproduce a trained `DecisionTreeRegressor`** entirely inside Power BI with DAX so users can pick parameters (Quantity, Category, Sub‑Category, PaymentMode) and see a **real‑time profit prediction**—no Python runtime, no gateway.

- **Model inputs (features)**: `Quantity`, `Category`, `Sub-Category`, `PaymentMode`  
- **Removed**: `Amount (Revenue)` (strongly correlated with profit → leakage risk)
- **Encoding**: integer codes identical to Python’s `LabelEncoder`/ordinal mapping  
- **Scaling**: `StandardScaler` z-scores applied in DAX
- **Model**: DecisionTreeRegressor reconstructed as `SWITCH(TRUE(), ...)`

---

## 2) Category mappings (fixed, must match training)
Use these as your dimension tables / slicers in Power BI (show names, pass codes):

```text
Category:
  Clothing -> 0
  Electronics -> 1
  Furniture -> 2

Sub-Category:
  Accessories -> 0
  Bookcases -> 1
  Chairs -> 2
  Electronic Games -> 3
  Furnishings -> 4
  Hankerchief -> 5
  Kurti -> 6
  Leggings -> 7
  Phones -> 8
  Printers -> 9
  Saree -> 10
  Shirt -> 11
  Skirt -> 12
  Stole -> 13
  T-shirt -> 14
  Tables -> 15
  Trousers -> 16

PaymentMode:
  COD -> 0
  Credit Card -> 1
  Debit Card -> 2
  EMI -> 3
  UPI -> 4
```

---

## 3) StandardScaler parameters (from Python)
Order: **[Quantity, Category, Sub‑Category, PaymentMode]**

```text
MEAN  = [3.76, 0.53333333, 7.83583333, 1.50833333]
SCALE = [2.20243804, 0.75755455, 4.46081636, 1.62529912]
```

DAX computes z-scores: `z = (x - mean) / scale` for each feature.

---

## 4) Decision tree (export_text from Python)
Tree thresholds are on **z-values**:

```text
|--- Sub-Category <= -1.42
|   |--- Quantity <= 3.06
|   |   |--- Quantity <= 0.34      ->  58.53
|   |   |--- Quantity >  0.34      -> 180.50
|   |--- Quantity >  3.06          -> -1307.00
|--- Sub-Category >  -1.42
|   |--- Quantity <= 3.97
|   |   |--- Sub-Category <= 1.72  ->  14.08
|   |   |--- Sub-Category >  1.72  ->  96.28
|   |--- Quantity >  3.97
|   |   |--- PaymentMode <= -0.62  ->  936.50
|   |   |--- PaymentMode >  -0.62  ->  151.50
```

---

## 5) Drop‑in DAX (single measure)
Create a measure named **ML Predicted Profit (No Amount)** and paste:

```DAX
ML Predicted Profit (No Amount) =
VAR Quantity_input = SELECTEDVALUE('pQuantity'[pQuantity Value])
VAR Category_input = SELECTEDVALUE('DimCategory'[Category Code])
VAR SubCat_input   = SELECTEDVALUE('DimSubCategory'[SubCategory Code])
VAR PayMode_input  = SELECTEDVALUE('DimPaymentMode'[PaymentMode Code])

-- StandardScaler (order: Quantity, Category, Sub-Category, PaymentMode)
VAR mu_Q = 3.76
VAR sc_Q = 2.20243804
VAR mu_C = 0.53333333
VAR sc_C = 0.75755455
VAR mu_S = 7.83583333
VAR sc_S = 4.46081636
VAR mu_P = 1.50833333
VAR sc_P = 1.62529912

-- z-scores
VAR Q = DIVIDE( Quantity_input - mu_Q, sc_Q )
VAR C = DIVIDE( Category_input - mu_C, sc_C )  -- not used in this tree; kept for future
VAR S = DIVIDE( SubCat_input   - mu_S, sc_S )
VAR P = DIVIDE( PayMode_input  - mu_P, sc_P )

-- Decision tree over z-values
RETURN
SWITCH(
    TRUE(),
    S <= -1.42 && Q <= 3.06 && Q <= 0.34,    58.53,
    S <= -1.42 && Q <= 3.06 && Q >  0.34,   180.50,
    S <= -1.42 && Q >  3.06,               -1307.00,

    S >  -1.42 && Q <= 3.97 && S <= 1.72,    14.08,
    S >  -1.42 && Q <= 3.97 && S >  1.72,    96.28,
    S >  -1.42 && Q >  3.97  && P <= -0.62,  936.50,
    S >  -1.42 && Q >  3.97  && P >  -0.62,  151.50,

    BLANK()
)
```
---

## 6) Quick test (Notebook ↔ Power BI parity)

Pick a test point in Python (e.g., `Quantity=8`, `Sub-Category="Phones"→8`, `PaymentMode="EMI"→3`).  
Compute the four z-scores with the MEAN/SCALE above and traverse the tree manually—your value must match the Power BI Card bound to.

If it deviates, confirm:
1. Encodings (codes) equal training mappings.
2. Slicer reading the **Code** column, not the Name.
3. Feature order for MEAN/SCALE is exactly `[Quantity, Category, Sub-Category, PaymentMode]`.

---

## 7) Why this approach
- Native Power BI inference (no Python/Gateway)
- Fully reproducible & explainable logic
- Easy to version / review in Git & in the PBIX
- All thresholds auditable (DAX = model card)

---

*End of in‑notebook notes.*
